### Import Necessary Libaries

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner

!conda install -c conda-forge geopy --yes # Uncomment out if you need to install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes # Uncomment out you need to install folium
import folium # plotting library

!pip install beautifulsoup4
!pip install lxml

from bs4 import BeautifulSoup

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-3.2.0               |           py36_0         770 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.3.1               |             py_0          25 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         868 KB

The following NEW packages will be INSTALLED:

    altair:  3.2.0-py36_0 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forg

After importing/installing all the necessary libraries/packages, we will be using BeautifulSoup to scrape the wiki page for the postal code table and convert it to a dataframe using pandas

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
results = requests.get(url)

In [3]:
# Web scraping tables with BeautifulSoup
soup = BeautifulSoup(results.content, 'lxml')
table = soup.find_all('table')[0]

# Reading table into dataframe
can_postal = pd.read_html(str(table))[0]
can_postal.columns = ['PostalCode', 'Borough', 'Neighborhood']
can_postal.head()
can_postal.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

Next we will delete all of the rows with 'Not assigned' listed as the 'Borough'

In [4]:
can_postal = can_postal.set_index('Borough')
can_postal = can_postal.drop('Not assigned', axis = 0).reset_index()
can_postal = can_postal[['PostalCode', 'Borough', 'Neighborhood']]
can_postal.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Since I changed the order of the columns, I need to reorder the columns to match the table shown in Coursera

In [5]:
# Removes 'Not assigned' values in the neighborhood column and copies the corresponding Borough name
can_postal['Neighborhood'].replace('Not assigned', can_postal['Borough'], inplace=True)
can_postal.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights
5,M6A,North York,Lawrence Manor
6,M7A,Queen's Park,Queen's Park
7,M9A,Etobicoke,Islington Avenue
8,M1B,Scarborough,Rouge
9,M1B,Scarborough,Malvern


In [6]:
# Groups the 'Neighborhood' column when there are multiple neighborhoods per PostalCode
can_postal = can_postal.groupby(['PostalCode','Borough'], sort = False).agg(lambda x: ', '.join(x)).reset_index()
can_postal.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [7]:
can_postal.columns

Index(['PostalCode', 'Borough', 'Neighborhood'], dtype='object')

In [8]:
can_postal.shape

(103, 3)

# -----------------------------------------------------------------------

### Adding Latitude and Longitude to the dataframe

Geocoder was not working for me, so I am uploading the csv file for the latitudes and longitudes.

In [9]:
# Loads the csv file to a df
loc_df = pd.read_csv('http://cocl.us/Geospatial_data')

# Renaming the Postal Code column to match the previous df
loc_df.columns = ['PostalCode', 'Latitude', 'Longitude']
loc_df.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [10]:
# Merges the two dataframes
can_loc = pd.concat([can_postal, loc_df], axis=1, sort=False)

In [11]:
can_loc.head()

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M1G,43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,M1H,43.773136,-79.239476


# -----------------------------------------------------------------------

## Explore and cluster the neighborhoods in Toronto


In [12]:
# Find the lat and long of Toronto
address = 'Toronto, CA'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [13]:
# create map of Toronto using latitude and longitude values from above
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map from 'can_loc' df
for lat, lng, borough, neighborhood in zip(can_loc['Latitude'], can_loc['Longitude'], can_loc['Borough'], can_loc['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [14]:
can_loc

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",M1E,43.763573,-79.188711
3,M6A,North York,"Lawrence Heights, Lawrence Manor",M1G,43.770992,-79.216917
4,M7A,Queen's Park,Queen's Park,M1H,43.773136,-79.239476
5,M9A,Etobicoke,Islington Avenue,M1J,43.744734,-79.239476
6,M1B,Scarborough,"Rouge, Malvern",M1K,43.727929,-79.262029
7,M3B,North York,Don Mills North,M1L,43.711112,-79.284577
8,M4B,East York,"Woodbine Gardens, Parkview Hill",M1M,43.716316,-79.239476
9,M5B,Downtown Toronto,"Ryerson, Garden District",M1N,43.692657,-79.264848


In [15]:
# View Borough's total postal codes
pd.value_counts(can_loc['Borough'].values, sort=False)

York                 5
Etobicoke           12
North York          24
Mississauga          1
Central Toronto      9
Queen's Park         1
West Toronto         6
East York            5
Downtown Toronto    18
East Toronto         5
Scarborough         17
dtype: int64

In [16]:
toronto_data = can_loc[can_loc['Borough'].str.contains('Toronto')].reset_index(drop = True)

In [17]:
toronto_data.head(10)

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",M1E,43.763573,-79.188711
1,M5B,Downtown Toronto,"Ryerson, Garden District",M1N,43.692657,-79.264848
2,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389
3,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975
4,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714
5,M5G,Downtown Toronto,Central Bay Street,M2R,43.782736,-79.442259
6,M6G,Downtown Toronto,Christie,M3A,43.753259,-79.329656
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",M3K,43.737473,-79.464763
8,M6H,West Toronto,"Dovercourt Village, Dufferin",M3L,43.739015,-79.506944
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",M4C,43.695344,-79.318389


### Define credentials

In [49]:
CLIENT_ID = 'UQA1MEJBGS4DSL2QU43GDK3FE3CC5121QVWZUWZUCYPOGWAS' # your Foursquare ID
CLIENT_SECRET = 'MFSQLKINWED3HSACNIR44SHO3QKF5BH42ISYT1VFJ5LKSYUX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + 'ENTERED')
print('CLIENT_SECRET:' + 'ENTERED')

Your credentails:
CLIENT_ID: ENTERED
CLIENT_SECRET:ENTERED


In [19]:
toronto_data.loc[0]

PostalCode                            M5A
Borough                  Downtown Toronto
Neighborhood    Harbourfront, Regent Park
PostalCode                            M1E
Latitude                          43.7636
Longitude                        -79.1887
Name: 0, dtype: object

In [20]:
radius = 500
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
print('URL entered with credentials')

URL entered with credentials


In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5d644826a87921002c91d1cf'},
 'response': {'venues': [{'id': '4f04779a02d5cce0cfc06151',
    'name': 'Chinese Visa Application Service Center',
    'location': {'address': '393 University Ave, Suite 1501',
     'crossStreet': 'in University Centre',
     'lat': 43.65402839343005,
     'lng': -79.38736458967021,
     'labeledLatLngs': [{'label': 'display',
       'lat': 43.65402839343005,
       'lng': -79.38736458967021}],
     'distance': 14,
     'cc': 'CA',
     'city': 'Toronto',
     'state': 'ON',
     'country': 'Canada',
     'formattedAddress': ['393 University Ave, Suite 1501 (in University Centre)',
      'Toronto ON',
      'Canada']},
    'categories': [{'id': '4bf58dd8d48988d126941735',
      'name': 'Government Building',
      'pluralName': 'Government Buildings',
      'shortName': 'Government',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/building/government_',
       'suffix': '.png'},
      'primary': True}],
 

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [23]:
toronto_venues = getNearbyVenues(names=toronto_data['Neighborhood'],
                                   latitudes=toronto_data['Latitude'],
                                   longitudes=toronto_data['Longitude']
                                  )

Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

### Examine the new dataframe

Now lets view the new dataframe and shape

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(783, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.763573,-79.188711,Swiss Chalet Rotisserie & Grill,43.767697,-79.189914,Pizza Place
1,"Harbourfront, Regent Park",43.763573,-79.188711,G & G Electronics,43.765309,-79.191537,Electronics Store
2,"Harbourfront, Regent Park",43.763573,-79.188711,Big Bite Burrito,43.766299,-79.190720,Mexican Restaurant
3,"Harbourfront, Regent Park",43.763573,-79.188711,Enterprise Rent-A-Car,43.764076,-79.193406,Rental Car Location
4,"Harbourfront, Regent Park",43.763573,-79.188711,Woburn Medical Centre,43.766631,-79.192286,Medical Center


In [25]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood']

In [26]:
# move neighborhood column to the first column
neighborhood = toronto_onehot['Neighborhood']
toronto_onehot.drop(labels=['Neighborhood'], axis=1,inplace = True)
toronto_onehot.insert(0, 'Neighborhood', neighborhood)
toronto_onehot

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,"Harbourfront, Regent Park",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,"Ryerson, Garden District",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [27]:
toronto_onehot.shape

(783, 206)

Adding the venues expanded the dataframe from (783, 7) to (783, 206)

We will not group the neighborhoods by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,"Adelaide, King, Richmond",0.333333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000
1,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.051282,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.025641
3,Business Reply Mail Processing Centre 969 Eastern,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.052632
5,"Cabbagetown, St. James Town",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000
6,Central Bay Street,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000
7,"Chinatown, Grange Park, Kensington Market",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.028571,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000
8,Christie,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000
9,Church and Wellesley,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000


Grouping the neighborhoods identified 38 unique values, decreasing the dataframe to 38 rows and it also automatically sorted the neighborhoods

Let's identify the top 3 venues in each neighborhood

In [29]:
num_top_venues = 3

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
      venue  freq
0   Airport  0.33
1      Park  0.33
2  Bus Stop  0.33


----Berczy Park----
           venue  freq
0      Cafeteria   0.5
1           Park   0.5
2  Movie Theater   0.0


----Brockton, Exhibition Place, Parkdale Village----
                 venue  freq
0                 Café  0.10
1          Coffee Shop  0.08
2  American Restaurant  0.05


----Business Reply Mail Processing Centre 969 Eastern----
               venue  freq
0        Pizza Place  0.25
1               Park  0.25
2  Mobile Phone Shop  0.25


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                venue  freq
0  Light Rail Station  0.11
1         Yoga Studio  0.05
2       Garden Center  0.05


----Cabbagetown, St. James Town----
                 venue  freq
0          Pizza Place  0.33
1  Empanada Restaurant  0.33
2           Food Truck  0.33


----Central Bay Street----
         venue  freq
0  Pizza 

#### Adding the data to a pandas dataframe

In [30]:
# Function sorts the values in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [31]:
# Identifies the top 7 venues for each neighborhood and outputs them in a df
num_top_venues = 7

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,"Adelaide, King, Richmond",Airport,Park,Bus Stop,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
1,Berczy Park,Cafeteria,Park,Yoga Studio,Diner,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
2,"Brockton, Exhibition Place, Parkdale Village",Café,Coffee Shop,American Restaurant,Bakery,Italian Restaurant,Yoga Studio,Diner
3,Business Reply Mail Processing Centre 969 Eastern,Park,Pizza Place,Bus Line,Mobile Phone Shop,Diner,Empanada Restaurant,Electronics Store
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Light Rail Station,Yoga Studio,Auto Workshop,Skate Park,Comic Shop,Smoke Shop,Restaurant


## Clustering the neighborhoods


Import the necessary packages for this section

In [32]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [33]:
# set number of clusters
kclusters = 3

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
toronto_grouped_clustering.head(10)

,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Art Gallery,Arts & Crafts Store,...,Toy / Game Store,Trail,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,0.333333,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0.000000,0.0,0.0,0.0,0.0,0.0,0.051282,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.025641
3,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.052632
5,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
6,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
7,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.028571,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
8,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
9,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


Here we are running the k-means clustering and using a random state so the results can be replicated

In [34]:
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 0, 0, 0, 0, 0, 2, 0], dtype=int32)

We are going to merge the new dataframe with the one we just created in the previous section. The merged dataframe should include Postal Code, Borough, Neighborhood, Lat, Long, the Clustering label just identified, and the top 7 venues for each neighborhood.

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [36]:
toronto_merged = toronto_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

In [37]:
toronto_merged.head()

,PostalCode,Borough,Neighborhood,PostalCode,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",M1E,43.763573,-79.188711,0,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Intersection,Mexican Restaurant,Rental Car Location
1,M5B,Downtown Toronto,"Ryerson, Garden District",M1N,43.692657,-79.264848,0,College Stadium,Skating Rink,General Entertainment,Café,Diner,Ethiopian Restaurant,Empanada Restaurant
2,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Sandwich Place,Bubble Tea Shop,Pizza Place
3,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975,0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Yoga Studio,Dog Run,Falafel Restaurant
4,M5E,Downtown Toronto,Berczy Park,M2L,43.757490,-79.374714,2,Cafeteria,Park,Yoga Studio,Diner,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant


We will not put all the values on a map of Toronto

In [39]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the clusters created

First cluster: Food & Drinks

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
0,Downtown Toronto,43.763573,-79.188711,0,Electronics Store,Breakfast Spot,Pizza Place,Medical Center,Intersection,Mexican Restaurant,Rental Car Location
1,Downtown Toronto,43.692657,-79.264848,0,College Stadium,Skating Rink,General Entertainment,Café,Diner,Ethiopian Restaurant,Empanada Restaurant
2,Downtown Toronto,43.799525,-79.318389,0,Chinese Restaurant,Fast Food Restaurant,Grocery Store,Pharmacy,Sandwich Place,Bubble Tea Shop,Pizza Place
3,East Toronto,43.786947,-79.385975,0,Japanese Restaurant,Chinese Restaurant,Bank,Café,Yoga Studio,Dog Run,Falafel Restaurant
5,Downtown Toronto,43.782736,-79.442259,0,Butcher,Grocery Store,Pharmacy,Pizza Place,Coffee Shop,Home Service,Discount Store
8,West Toronto,43.739015,-79.506944,0,Grocery Store,Hotel,Shopping Mall,Bank,Park,General Travel,General Entertainment
9,Downtown Toronto,43.695344,-79.318389,0,Skating Rink,Cosmetics Shop,Beer Store,Spa,Athletics & Sports,Curling Ice,Park
10,West Toronto,43.676357,-79.293031,0,Pub,Health Food Store,Trail,Dessert Shop,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
11,East Toronto,43.679557,-79.352188,0,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Furniture / Home Store,Bubble Tea Shop,Indian Restaurant
12,Downtown Toronto,43.668999,-79.315572,0,Park,Board Shop,Sushi Restaurant,Sandwich Place,Liquor Store,Burger Joint,Burrito Place


Second cluster: Group activities

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
35,Downtown Toronto,43.724766,-79.532242,1,Baseball Field,Yoga Studio,Field,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant


Third cluster: Park/Travel

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(4, toronto_merged.shape[1]))]]

,Borough,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue
4,Downtown Toronto,43.757490,-79.374714,2,Cafeteria,Park,Yoga Studio,Diner,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant
6,Downtown Toronto,43.753259,-79.329656,2,Park,Food & Drink Shop,Farmers Market,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
7,Downtown Toronto,43.737473,-79.464763,2,Airport,Park,Bus Stop,Falafel Restaurant,Ethiopian Restaurant,Empanada Restaurant,Electronics Store
23,Central Toronto,43.689026,-79.453512,2,Park,Market,Fast Food Restaurant,Women's Store,Gourmet Shop,Falafel Restaurant,Empanada Restaurant
25,Central Toronto,43.713756,-79.490074,2,Park,Construction & Landscaping,Basketball Court,Bakery,Discount Store,Falafel Restaurant,Ethiopian Restaurant


After using different clustering values (5, 3, and 2), I found 3 gave the best breakdown. When I used 5, three clusters only had 1 venue. When I used two, all of the venues in cluster 3 in this notebook moved to the first cluster. By using 3 as the number of clusters, I was able to break away 6 from the main grouping. Also, the 5 clusters overlapped a lot. By using 3 clusters, it gave a more definite separation between the groupings.